In [1]:
#!pip install SpeechRecognition

### Google Web Speech API

In [5]:
import os
import csv
import re
import speech_recognition as sr
import pandas as pd

In [6]:
# Папка с аудиофайлами
#folder_path = "C:/Users/Anna/Downloads/audio"
# Путь к выходному CSV файлу
#output_csv_file = "C:/Users/Anna/Downloads/audio_text/output.csv"

# Папка с аудиофайлами
folder_path = output_folder

# Путь к выходному CSV файлу
output_csv_file = os.path.join(base_dir, "output.csv")

# Создаем экземпляр Recognizer
recognizer = sr.Recognizer()

# Функция для извлечения числа из имени файла
def extract_number(filename):
    # Используем регулярное выражение для извлечения числа из имени файла
    match = re.search(r'\d+', filename)
    if match:
        return int(match.group())
    else:
        return float('inf')  # Если числа нет, возвращаем бесконечность

# Получаем список файлов и сортируем его на основе извлеченных чисел
files = sorted(os.listdir(folder_path), key=lambda x: extract_number(x))

# Открываем CSV файл для записи
with open(output_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Название файла', 'Распознанный текст'])  # Записываем заголовки столбцов

    # Обрабатываем каждый файл в отсортированном списке
    for filename in files:
        if filename.endswith('.wav'):  # Проверяем, что файл имеет расширение .wav
            audio_file = os.path.join(folder_path, filename)

            # Открываем аудиофайл с помощью AudioFile
            with sr.AudioFile(audio_file) as source:
                audio_data = recognizer.record(source)  # Записываем аудиофайл в AudioData объект

                # Используем Google Web Speech API для распознавания на английском языке
                try:
                    #text = recognizer.recognize_google(audio_data, language="en-US")  # Распознавание на английском языке
                    text = recognizer.recognize_google(audio_data, language="en-US")
                    csv_writer.writerow([filename, text])  # Записываем результаты в CSV файл
                except sr.UnknownValueError:
                    csv_writer.writerow([filename, "Речь не распознана"])  # Записываем информацию об ошибке
                except sr.RequestError as e:
                    csv_writer.writerow([filename, f"Ошибка запроса к API: {e}"])  # Записываем информацию об ошибке

print(f"Результаты распознавания сохранены в файле: {output_csv_file}")


Результаты распознавания сохранены в файле: C:/Users/Anna/Downloads/audio_text/output.csv


In [7]:
data = pd.read_csv(output_csv_file, sep=',')
data

,Название файла,Распознанный текст
0,chunk0.wav,Речь не распознана
1,chunk1.wav,Речь не распознана
2,chunk2.wav,Речь не распознана
3,chunk3.wav,Речь не распознана
4,chunk10.wav,Речь не распознана
5,chunk12.wav,Речь не распознана
6,chunk22.wav,Речь не распознана
7,chunk99.wav,Речь не распознана
8,chunk100.wav,happy Sunday


### CMU Sphinx

In [1]:
import os
import csv
import pandas as pd
import speech_recognition as sr

In [3]:
# Создаем распознаватель речи
recognizer = sr.Recognizer()

# Путь к папке с аудиофайлами и к выходному CSV-файлу
folder_path = "C:/Users/Anna/Downloads/audio"
output_csv_file = "C:/Users/Anna/Downloads/audio_text/output_1.csv"

# Функция для преобразования аудиофайла в текст с использованием CMU Sphinx
def transcribe_audio_with_sphinx(audio_path):
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_sphinx(audio_data)
            return text
        except sr.UnknownValueError:
            return "CMU Sphinx не смогло распознать аудио"
        except sr.RequestError as e:
            return f"Ошибка запроса к CMU Sphinx: {e}"

# Список для хранения результатов
results = []

# Перебираем все файлы в папке
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if file_name.lower().endswith('.wav'):  # Обрабатываем только файлы с расширением .wav
        print(f"Обрабатываем файл: {file_name}")
        transcription = transcribe_audio_with_sphinx(file_path)
        results.append([file_name, transcription, "CMU Sphinx"])

# Создаем DataFrame и сохраняем в CSV
df = pd.DataFrame(results, columns=["File Name", "Transcription", "Model"])
df.to_csv(output_csv_file, index=False)
print(f"Результаты сохранены в {output_csv_file}")

Обрабатываем файл: chunk0.wav
Обрабатываем файл: chunk1.wav
Обрабатываем файл: chunk10.wav
Обрабатываем файл: chunk100.wav
Обрабатываем файл: chunk12.wav
Обрабатываем файл: chunk2.wav
Обрабатываем файл: chunk22.wav
Обрабатываем файл: chunk3.wav
Обрабатываем файл: chunk99.wav
Результаты сохранены в C:/Users/Anna/Downloads/audio_text/output_1.csv


### Google Web Speech API & CMU Sphinx

In [ ]:
import os
import csv
import pandas as pd
import speech_recognition as sr

# Путь к папке с аудиофайлами и к папке для выходных CSV файлов
folder_path = "C:/Users/Anna/Downloads/audio"
output_folder = "C:/Users/Anna/Downloads/audio_text/"

# Создаем экземпляр Recognizer
recognizer = sr.Recognizer()

# Функция для преобразования аудиофайла в текст с использованием Google Web Speech API
def transcribe_audio_with_google(audio_path):
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language="en-US")  # Распознавание на английском языке
            return text
        except sr.UnknownValueError:
            return "Speech Recognition could not understand audio"
        except sr.RequestError as e:
            return f"Request error from Google API: {e}"

# Функция для преобразования аудиофайла в текст с использованием CMU Sphinx
def transcribe_audio_with_sphinx(audio_path):
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_sphinx(audio_data)
            return text
        except sr.UnknownValueError:
            return "CMU Sphinx could not understand audio"
        except sr.RequestError as e:
            return f"Request error from CMU Sphinx: {e}"

# Создаем папку для выходных файлов, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Списки для хранения результатов
google_results = []
sphinx_results = []

# Перебираем все файлы в папке
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if file_name.lower().endswith('.wav'):  # Обрабатываем только файлы с расширением .wav
        print(f"Processing file: {file_name}")

        # Прогоняем файл через Google Web Speech API
        google_transcription = transcribe_audio_with_google(file_path)
        google_results.append([file_name, google_transcription])

        # Прогоняем файл через CMU Sphinx
        sphinx_transcription = transcribe_audio_with_sphinx(file_path)
        sphinx_results.append([file_name, sphinx_transcription])

# Сортируем результаты по цифровой части имени файла
def extract_number(file_name):
    numbers = ''.join(filter(str.isdigit, file_name))
    return int(numbers) if numbers else 0

google_results.sort(key=lambda x: extract_number(x[0]))
sphinx_results.sort(key=lambda x: extract_number(x[0]))

# Записываем результаты в файлы
google_csv_file = os.path.join(output_folder, 'Google_Web_Speech_API.csv')
sphinx_csv_file = os.path.join(output_folder, 'CMU_Sphinx.csv')

# Записываем результаты в файл для Google Web Speech API
with open(google_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['File Name', 'Transcription'])  # Записываем заголовки столбцов
    csv_writer.writerows(google_results)

# Записываем результаты в файл для CMU Sphinx
with open(sphinx_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['File Name', 'Transcription'])  # Записываем заголовки столбцов
    csv_writer.writerows(sphinx_results)

print(f"Results saved to files: {google_csv_file} and {sphinx_csv_file}")
